In [1]:
from pyparsing import Word, alphas, nums, Forward, infixNotation, opAssoc, CaselessKeyword, one_of, oneOf

In [2]:
start_conditions = ["NOW > 78909", "N > 5", "now >= 374343 AND n > 5"]
stop_conditions = ["NOW > 78909", "NOT N < 5", "now >= 374343 AND n < 5"]

In [3]:
# Define basic elements (operands)
identifier = oneOf(["NOW", "N"], caseless=True)
number = Word(nums)
operand = identifier | number

# Define operators
comparison_operator = oneOf("< <= > >=")
and_operator = CaselessKeyword("AND")
or_operator = CaselessKeyword("OR")
not_operator = CaselessKeyword("NOT")

# Grammar definition using infixNotation
expression = infixNotation(
    operand,
    [
        (comparison_operator, 2, opAssoc.LEFT),  # Comparison operators
        (not_operator, 1, opAssoc.RIGHT),  # NOT operator - unary, right associative
        (and_operator, 2, opAssoc.LEFT),  # AND operator
        (or_operator, 2, opAssoc.LEFT),  # OR operator
    ],
)

In [14]:
def eval_expression(parsed, var_values):
    if isinstance(parsed, str):
        if parsed in var_values:
            return var_values[parsed]
        else:
            # Assume it's a literal number
            return float(parsed)
    else:
        if len(parsed) == 1:
            return eval_expression(parsed[0], var_values)
        elif len(parsed) == 2:
            assert parsed[0] == "NOT"
            return not eval_expression(parsed[1], var_values)
        elif len(parsed) == 3:
            op = parsed[1]
            if op in ["<", "<=", ">", ">="]:
                # Comparison operators
                left = eval_expression(parsed[0], var_values)
                right = eval_expression(parsed[2], var_values)
                return eval(f"{left} {op} {right}")
            elif op in ["AND", "OR"]:
                # Logical operators
                left = eval_expression(parsed[0], var_values)
                right = eval_expression(parsed[2], var_values)
                if op == "AND":
                    return left and right
                elif op == "OR":
                    return left or right
        else:
            raise NotImplementedError()

In [15]:
for expr in start_conditions + stop_conditions:
    parsed_expr = expression.parseString(expr)
    print(parsed_expr.asList())
    print(eval_expression(parsed_expr, {'NOW': 923120, 'N': 3}))

[['NOW', '>', '78909']]
True
[['N', '>', '5']]
False
[[['NOW', '>=', '374343'], 'AND', ['N', '>', '5']]]
False
[['NOW', '>', '78909']]
True
[['NOT', ['N', '<', '5']]]
False
[[['NOW', '>=', '374343'], 'AND', ['N', '<', '5']]]
True


In [16]:
pl = parsed_expr.asList()

In [12]:
type(pl[0][0][2])

str